# Final Project:
## New Facility Location Selection
### by: Jeffrey Dupree

This notebook will scrape neighborhood information from a ZIP-CODES.COM page https://www.zip-codes.com/state/fl.asp#zipcodes to create a dataframe consisting of the Zip Code, the City name, County name and the Zip Code type.

#### Section One: Scrape Tampa, FL ZIP Codes from website

First, we install the necessary libraries.

In [1]:
# If you don't have these packages available, uncomment the appropriate lines below to install them.

import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Next, we need to get the information from the webpage using `requests.get`.

In [2]:
source = requests.get('https://www.zip-codes.com/state/fl.asp#zipcodes').text

Use the BeautifulSoup package to scrape the information from the webpage. I used the lxml parsing method, but you can use any you like.

In [3]:
soup = BeautifulSoup(source, 'lxml')

Find the table using `soup.find` from BeautifulSoup. Uncomment the second line to see the structure and content of the table. The tags are needed for the next steps.

In [4]:
table = soup.find(id="tblZIP")
# print(table.prettify())

Now a pandas dataframe needs to be created. This will require looping through the elements from the table and assigning the to a list. The list can then be made into a dataframe using `pd.DataFrame`. The columns will need header names. I manually assigned these instead of pulling them from the BeautifulSoup object `table`.

In [5]:
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

# Label the columns.
df = pd.DataFrame(res[1:], columns=['Zip_Code','City','County','Type'])

# Remove the text 'Zip Code' from the records in the Zip Code column.
df['Zip_Code'] = df['Zip_Code'].str[-5:]

# Select only the Zip Codes for Tampa, FL.
df = df.loc[df['City'] == "Tampa"]

Next remove the rows where the type is "P.O. Box".

In [6]:
# Remove rows with Type = "P.O. Box" and "Unique", and reset the index to start at 0
df = df[df.Type == 'Standard']
df = df.reset_index(drop=True)

The resulting dataframe looks like this.

In [7]:
df

,Zip_Code,City,County,Type
0,33602,Tampa,Hillsborough,Standard
1,33603,Tampa,Hillsborough,Standard
2,33604,Tampa,Hillsborough,Standard
3,33605,Tampa,Hillsborough,Standard
4,33606,Tampa,Hillsborough,Standard
5,33607,Tampa,Hillsborough,Standard
6,33609,Tampa,Hillsborough,Standard
7,33610,Tampa,Hillsborough,Standard
8,33611,Tampa,Hillsborough,Standard
9,33612,Tampa,Hillsborough,Standard


Check the size of the dataframe.

In [8]:
df.shape

(26, 4)

#### Section Two: Geolocate ZIP Codes

In [9]:
# @hidden_cell
user_agent = "JGD_20191006"

In [10]:
import re

# Uncomment next line to install geopy if necessary.
!{sys.executable} -m pip install geopy

from tqdm import tqdm #This will allow a progress bar to show that there is progress being made. This is helpful when an
tqdm.pandas()         #iterative process may take more than a few seconds.

from functools import partial #This will allow multiple arguments to be passed to RateLimiter.

import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent=user_agent)
from geopy.extra.rate_limiter import RateLimiter #This will get around getting shut down for too many request errors.
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.5, max_retries=2, error_wait_seconds=5.0, swallow_exceptions=True, return_value_on_exception=None)
df['location'] = df['Zip_Code'].progress_apply(partial(geocode, country_codes='us'))

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

C:\Users\JeffDupree\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  0%|                                                                                           | 0/26 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:26<00:00,  1.02s/it]


,Zip_Code,City,County,Type,location,point
0,33602,Tampa,Hillsborough,Standard,"(Ybor City, Tampa, Hillsborough County, Florid...","(27.9516574, -82.449638, 0.0)"
1,33603,Tampa,Hillsborough,Standard,"(Tampa, Hillsborough County, Florida, 33603, U...","(27.9824664343203, -82.4630092024596, 0.0)"
2,33604,Tampa,Hillsborough,Standard,"(Sulphur Springs, Tampa, Hillsborough County, ...","(28.0127051, -82.4665599, 0.0)"
3,33605,Tampa,Hillsborough,Standard,"(East Ybor, Tampa, Hillsborough County, Florid...","(27.956726, -82.405085, 0.0)"
4,33606,Tampa,Hillsborough,Standard,"(Davis Islands, Tampa, Hillsborough County, Fl...","(27.9368959, -82.4596737, 0.0)"


In [11]:
df[['Latitude','Longitude','3']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df = df.drop(columns=['point','3'])

Now there are latitude and longitude values for each of the postal codes.

In [12]:
df['location'] = df['location'].astype(str)
df['location'] = df['location'].str.split(", ").str[0].tolist()
df = df.rename(columns={"location": "Neighborhood"})

In [13]:
df.head()

,Zip_Code,City,County,Type,Neighborhood,Latitude,Longitude
0,33602,Tampa,Hillsborough,Standard,Ybor City,27.951657,-82.449638
1,33603,Tampa,Hillsborough,Standard,Tampa,27.982466,-82.463009
2,33604,Tampa,Hillsborough,Standard,Sulphur Springs,28.012705,-82.466560
3,33605,Tampa,Hillsborough,Standard,East Ybor,27.956726,-82.405085
4,33606,Tampa,Hillsborough,Standard,Davis Islands,27.936896,-82.459674


#### Section Three: Locate Zip Codes Lacking Gyms

In [14]:
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Uncomment this line if you haven't installed folium for accessing the Foursquare API
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [15]:
# Create map of Tampa using latitude and longitude values
tampa = geolocator.geocode({"state": "fl", "city": "tampa"})
map_tampa = folium.Map(location=[tampa.latitude, tampa.longitude], zoom_start=10)

# add markers to map
for lat, lng, county, city in zip(df['Latitude'], df['Longitude'], df['County'], df['City']):
    label = '{}, {}'.format(county, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tampa)  
    
map_tampa

In [16]:
# @hidden_cell
CLIENT_ID = 'MAI43NUPMV0YXXNFKS2XVGUPBMIB5SBO5T5W5FV4ZND2VTJW' # your Foursquare ID
CLIENT_SECRET = 'V1POSAELWQ0NIURPOW2C43LH2FTO5NJ0VGYQXMSD2OGRLEND' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

Create the url that will query the Foursquare API for the top 100 venues within 500 meters of the location. The cell above assigns the client ID and client secret to variables that will be called below.

In [17]:
search_lat = df.Latitude[0]
search_lon = df.Longitude[0]
LIMIT = 150 # Limit of number of venues returned by Foursquare API
radius = 1000 # Define radius in meters

# Create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    search_lat, 
    search_lon, 
    radius, 
    LIMIT)


In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dabca94d0a149002c45ee81'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Village of Tampa',
  'headerFullLocation': 'Village of Tampa, University',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 27.960657409000007,
    'lng': -82.43946844978649},
   'sw': {'lat': 27.94265739099999, 'lng': -82.4598075502135}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b53ccf9f964a520b8ab27e3',
       'name': 'Pour House at Grand Central',
       'location': {'address': '1208 E Kennedy Blvd #112',
        'crossStreet': 'at N. 11th St.',
        'lat': 27.95

In [19]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # Flatten JSON

# Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pour House at Grand Central,Bar,27.951357,-82.447740
1,Crunch - Channelside,Gym / Fitness Center,27.951152,-82.447940
2,Cena,Italian Restaurant,27.951569,-82.447869
3,Publix - Channelside,Grocery Store,27.952128,-82.448741
4,City Dog Cantina,Mexican Restaurant,27.951118,-82.447726


This creates a function for using the Foursquare API to find the nearby venues for all of the boroughs in the dataframe.

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip_Code', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
tampa_venues = getNearbyVenues(names=df['Zip_Code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

33602
33603
33604
33605
33606
33607
33609
33610
33611
33612
33613
33614
33615
33616
33617
33618
33619
33621
33624
33625
33626
33629
33634
33635
33637
33647


In [23]:
print(tampa_venues.shape)
tampa_venues.head()

(664, 7)


,Zip_Code,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,33602,27.951657,-82.449638,Pour House at Grand Central,27.951357,-82.447740,Bar
1,33602,27.951657,-82.449638,Crunch - Channelside,27.951152,-82.447940,Gym / Fitness Center
2,33602,27.951657,-82.449638,Cena,27.951569,-82.447869,Italian Restaurant
3,33602,27.951657,-82.449638,Publix - Channelside,27.952128,-82.448741,Grocery Store
4,33602,27.951657,-82.449638,City Dog Cantina,27.951118,-82.447726,Mexican Restaurant


In [24]:
tampa_venues['Venue Category'].unique()

array(['Bar', 'Gym / Fitness Center', 'Italian Restaurant',
       'Grocery Store', 'Mexican Restaurant', 'Food Truck', 'Dog Run',
       'Art Gallery', 'Theater', 'Plaza', 'Pizza Place', 'Tea Room',
       'Coffee Shop', 'Beer Garden', 'Asian Restaurant', 'Bagel Shop',
       'Skate Park', 'Movie Theater', 'Café', 'Speakeasy',
       'Tex-Mex Restaurant', 'Harbor / Marina', 'Restaurant',
       'Floating Market', 'Hockey Arena', 'Print Shop', 'Aquarium',
       'Sports Bar', 'Hotel', 'Cuban Restaurant',
       'Mediterranean Restaurant', 'Deli / Bodega', 'Zoo Exhibit',
       'BBQ Joint', 'Shipping Store', 'Park', 'Thai Restaurant',
       'Gastropub', 'Nightclub', 'Gym', 'Bowling Alley', 'Juice Bar',
       'French Restaurant', 'Cocktail Bar', 'Caribbean Restaurant',
       'Sandwich Place', 'Clothing Store', 'Bank', 'Pharmacy',
       'Chinese Restaurant', 'American Restaurant', 'Hookah Bar',
       'Fast Food Restaurant', 'Cruise', 'Bus Station', 'Port',
       'Burger Joint', 'Bre

As you can see above, there are several venue categories that could be generally categorized as a 'gym'. There are other venue categories that are not necesarily types of gyms, but might compete with a gym as a place where people go to be active. Another venue that would compete with a gym is 'Military Base'. Military bases have gyms and fitness centers for military members at no cost. We will need to recode these categories with a common category name (i.e., gym).

In [25]:
gym = ['Gym / Fitness Center', 'Park', 'Martial Arts Dojo', 'Gym', 'Pool', 'Tennis Court', 'Disc Golf', 'Volleyball Court',
       'Soccer Field', 'Basketball Court', 'Yoga Studio', 'College Basketball Court', 'College Gym','College Track',
       'Dance Studio', 'Military Base', 'Athletics & Sports', 'Golf Course', 'Baseball Field', 'Trail', 'Hockey Arena',
       'Hockey Field', 'Track', 'Water Park', 'Outdoors & Recreation', 'State / Provincial Park', 'Playground']

In [26]:
tampa_venues['Venue Category'].replace(to_replace =gym, value ="Gym", inplace=True)

In [27]:
tampa_venues.groupby('Zip_Code').count()

,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip_Code,,,,,,
33602,89,89,89,89,89,89
33603,19,19,19,19,19,19
33604,41,41,41,41,41,41
33605,18,18,18,18,18,18
33606,62,62,62,62,62,62
33607,5,5,5,5,5,5
33609,19,19,19,19,19,19
33610,14,14,14,14,14,14
33611,5,5,5,5,5,5


In [28]:
print('There are {} uniques categories.'.format(len(tampa_venues['Venue Category'].unique())))

There are 172 uniques categories.


We use one-hot encoding to determine if a venue type exists in a neighborhood. This will create a column for each of the unique categories, and assign a value of 1 if that venue type exists in the neighborhood or 0 otherwise.

In [29]:
# One hot encoding
tampa_onehot = pd.get_dummies(tampa_venues[['Venue Category']], prefix="", prefix_sep="")

# Add zip code column back to dataframe
tampa_onehot['Zip_Code'] = tampa_venues['Zip_Code'] 

# Move zip code column to the first column
fixed_columns = [tampa_onehot.columns[-1]] + list(tampa_onehot.columns[:-1])
tampa_onehot = tampa_onehot[fixed_columns]

tampa_onehot.head()

,Zip_Code,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,...,Tree,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,33602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,33602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
tampa_onehot.shape

(664, 173)

With the one-hot encoded data, we can determine the frequency with which each venue type occurs in each borough. This results in a dataframe with a column for each unique venue type and a row for each unique borough.

In [31]:
tampa_grouped = tampa_onehot.groupby('Zip_Code').mean().reset_index()
tampa_grouped.head()

,Zip_Code,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,...,Tree,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,33602,0.0,0.0,0.0,0.0,0.0,0.022472,0.000000,0.011236,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.011236
1,33603,0.0,0.0,0.0,0.0,0.0,0.000000,0.052632,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000
2,33604,0.0,0.0,0.0,0.0,0.0,0.048780,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.02439,0.268293
3,33605,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000
4,33606,0.0,0.0,0.0,0.0,0.0,0.032258,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.0,0.00000,0.000000


Next we will determine the five most frequent venues within a borough to describe a neighborhood 'type', and group the borough by type symilarity.

In [32]:
num_top_venues = 5

for hood in tampa_grouped['Zip_Code']:
    print("----"+hood+"----")
    temp = tampa_grouped[tampa_grouped['Zip_Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----33602----
         venue  freq
0          Bar  0.08
1          Gym  0.07
2  Coffee Shop  0.04
3         Café  0.04
4        Hotel  0.03


----33603----
          venue  freq
0           Gym  0.16
1  Intersection  0.11
2   Art Gallery  0.11
3       Brewery  0.05
4      Pharmacy  0.05


----33604----
                          venue  freq
0                   Zoo Exhibit  0.27
1                           Gym  0.12
2  Theme Park Ride / Attraction  0.10
3                           Bar  0.05
4           American Restaurant  0.05


----33605----
                  venue  freq
0   Fried Chicken Joint  0.11
1        Sandwich Place  0.11
2  Fast Food Restaurant  0.11
3        Hardware Store  0.06
4     Convenience Store  0.06


----33606----
         venue  freq
0          Gym  0.18
1  Coffee Shop  0.06
2   Sports Bar  0.05
3        Hotel  0.05
4  Pizza Place  0.05


----33607----
                     venue  freq
0               Food Truck   0.2
1          Harbor / Marina   0.2
2              

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Zip_Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
zip_venues_sorted = pd.DataFrame(columns=columns)
zip_venues_sorted['Zip_Code'] = tampa_grouped['Zip_Code']

for ind in np.arange(tampa_grouped.shape[0]):
    zip_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tampa_grouped.iloc[ind, :], num_top_venues)

zip_venues_sorted.head()

,Zip_Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,33602,Bar,Gym,Café,Coffee Shop,Pizza Place
1,33603,Gym,Intersection,Art Gallery,Sporting Goods Shop,Grocery Store
2,33604,Zoo Exhibit,Gym,Theme Park Ride / Attraction,American Restaurant,Bar
3,33605,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Convenience Store,Grocery Store
4,33606,Gym,Coffee Shop,Bar,Hotel,Pizza Place


Now that we can see what the five most common venues are in each Zip Code are, we can eliminate those Zip Codes with 'gym' type venues in the top five.

In [35]:
zip_venues_reduced = zip_venues_sorted[(zip_venues_sorted['1st Most Common Venue'] != 'Gym') & (zip_venues_sorted['2nd Most Common Venue'] != 'Gym') & 
                                       (zip_venues_sorted['3rd Most Common Venue'] != 'Gym') & (zip_venues_sorted['4th Most Common Venue'] != 'Gym') &
                                       (zip_venues_sorted['5th Most Common Venue'] != 'Gym')]
zip_venues_reduced.shape

(5, 6)

Now the list only includes Zip Codes where 'gym' type venues are not one of the five most frequent venue types.

In [36]:
index = zip_venues_reduced.index
locations = tampa_grouped[['Zip_Code','Gym']].iloc[index].sort_values(by=['Gym'], ascending=True)
locations

,Zip_Code,Gym
3,33605,0.000000
5,33607,0.000000
18,33625,0.000000
7,33610,0.071429
17,33624,0.111111


In [37]:
cols = ['Zip_Code']
locations = locations.join(df.set_index(cols), on=cols)
locations = locations.rename(columns={"Gym": "Gym Frequency"}).reset_index(drop=True)
locations

,Zip_Code,Gym Frequency,City,County,Type,Neighborhood,Latitude,Longitude
0,33605,0.000000,Tampa,Hillsborough,Standard,East Ybor,27.956726,-82.405085
1,33607,0.000000,Tampa,Hillsborough,Standard,Tampa,27.973055,-82.588636
2,33625,0.000000,Tampa,Hillsborough,Standard,Hillsborough County,28.068451,-82.557170
3,33610,0.071429,Tampa,Hillsborough,Standard,Ybor City,27.977944,-82.442975
4,33624,0.111111,Tampa,Hillsborough,Standard,Hillsborough County,28.071590,-82.525613


Now we can display the locations on a map. Selecting a marker on the map will display that zip code and the frequency of 'gym' type venues within 1km of the zip code central point.

In [38]:
# Create map
map_locations = folium.Map(location=[tampa.latitude, tampa.longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(locations.shape[0])
ys = [i + x + (i*x)**2 for i in range(locations.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
count = 0
for lat, lon, poi, freq, nbh in zip(locations['Latitude'], locations['Longitude'], locations['Zip_Code'], 
                                    locations['Gym Frequency'], locations['Neighborhood']):
    label = folium.Popup('Neigborhood: ' + str(nbh) + ' / Zip Code: ' + str(poi) + ' / Gym Frequency: ' + str(freq), 
                         parse_html=False,)
    count = count + 1
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[count-1],
        fill=True,
        fill_color=rainbow[count-1],
        fill_opacity=0.7).add_to(map_locations)
       
map_locations

In [39]:
import urllib, json
url = 'https://opendata.arcgis.com/datasets/d356e19e0fb34524b54d189fafb0d675_0.geojson'
with urllib.request.urlopen(url) as url:
    plt = json.loads(url.read().decode())
plt

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'Zip_Code': '33860',
    'Name': 'Mulberry',
    'GlobalID': '4eeb9b57-0000-40f5-9f03-daf1b828c613',
    'Shape__Area': 5293244.08709717,
    'Shape__Length': 9059.66346952034},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-82.0549390992785, 27.7265117677886],
      [-82.0607591683534, 27.7266736634492],
      [-82.0611897889828, 27.7265201678582],
      [-82.061361599104, 27.725676908574],
      [-82.0611876491316, 27.7226875202018],
      [-82.0612726302737, 27.7206178510651],
      [-82.0611001436625, 27.7202346623123],
      [-82.060755428975, 27.7199282020326],
      [-82.0600662153963, 27.7196985438136],
      [-82.0591117175688, 27.7192739636024],
      [-82.0573097094845, 27.7193931005122],
      [-82.054926540895, 27.7192894934391],
      [-82.0549314595998, 27.7237459779917],
      [-82.0549390992785, 27.7265117677886]]]}},
  {'type': 'Feature',
   'properties': {'

In [40]:
!{sys.executable} -m pip install geojson
import geojson
from geojson import Feature, Point, FeatureCollection, MultiPolygon

In [41]:
locations

,Zip_Code,Gym Frequency,City,County,Type,Neighborhood,Latitude,Longitude
0,33605,0.000000,Tampa,Hillsborough,Standard,East Ybor,27.956726,-82.405085
1,33607,0.000000,Tampa,Hillsborough,Standard,Tampa,27.973055,-82.588636
2,33625,0.000000,Tampa,Hillsborough,Standard,Hillsborough County,28.068451,-82.557170
3,33610,0.071429,Tampa,Hillsborough,Standard,Ybor City,27.977944,-82.442975
4,33624,0.111111,Tampa,Hillsborough,Standard,Hillsborough County,28.071590,-82.525613


Using the GeoJSON file from https://opendata.arcgis.com/datasets/d356e19e0fb34524b54d189fafb0d675_0.geojson polygons for the Zip Codes of intereset can be defined using the latitude and longitude coordinates. Below we create a list of coordinates for both latitudes and longitudes, then place these lists at the end of the dataframe.

In [42]:
for i in reversed(range(len(plt['features']))):
    count = locations.shape[0]
    for j in range(locations.shape[0]):
        if plt['features'][i]['properties']['Zip_Code'] != locations['Zip_Code'][j]:
            count = count - 1
            if count == 0:
                del plt['features'][i]
                break

Now the polygons for the areas represented by the zip code can be overlaid on the map.

In [47]:
# Create map
map_test = folium.Map(location=[tampa.latitude, tampa.longitude], zoom_start=11)

# Add polygons to the map
for i in range(len(plt['features'])):
    neighborhood = locations.Neighborhood[i]
    zip_code = locations.Zip_Code[i]
    geojson = folium.GeoJson(
        plt['features'][i],
        name=neighborhood
    )
    popup = folium.Popup(neighborhood + " " + zip_code)
    popup.add_to(geojson)
    geojson.add_to(map_test)

folium.LayerControl().add_to(map_test)

map_test

In [54]:
print(locations.Neighborhood[3] + " " + locations.Zip_Code[3])
plt['features'][3]['properties']['Zip_Code']

Ybor City 33610


'33625'